<a href="https://colab.research.google.com/github/exglade/query-anime/blob/main/query_anime_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Query Anime List

## Problem Statement

I have a spreadsheet of anime list with following columns:

- **Parent**: The anime's parent title.
- **Title**: The full anime title.
- **Genre**: The anime genre (according to MyAnimeList).
- **Season**: The xth season of the anime. Non-season type: OVA, Movie, ONA, Special.
- **Total Episodes**: The total number of episodes that the anime has.
- **Date of Release**: The date when the anime start airing.
- **Date of Completion**: The date when the anime finishes airing.
- **Status**: The airing status of the anime: Unreleased, Ongoing, Released.
- **Last**: The last episode I've watched.
- **Score**: My personal rating of the anime on a scale of 0-5.

There is a list of Anime tabs stored in OneTab. The list can be exported from OneTab into text. Examples below:

```text
https://www.microsoft.com/design/fluent/#/ | Microsoft Design
https://medium.com/microsoft-design/designing-for-power-simplicity-9cddec615567 | Designing for Power and Simplicity - Microsoft Design - Medium
...
```

After clean-up, there are 150 anime in the list. Performing query on anime one by one and copy the information would take hours.

There has to be a better way so it would be easier for me in future too! 😁

## Solution

1. Data: Extract, Transform, Load the anime list.
2. Search for anime by name.
3. Map anime to anime list's identifier.
4. Fetch Anime details.
5. Export into spreadsheet. 🎉

### API
- [MyAnimeList](https://myanimelist.net/) - [v2](https://myanimelist.net/apiconfig/references/api/v2) | [Authorisation guide](https://myanimelist.net/blog.php?eid=835707)
- [AniList](https://anilist.co/) - [v2](https://anilist.gitbook.io/anilist-apiv2-docs/)
- [Jikan](https://jikan.moe/) - [v3](https://jikan.docs.apiary.io/)

# Data: Extract, Transform, Load

1. Export the list of tabs using OneTab export
2. For each line, filter away non-relevant lines and parse the data.
3. Save the parsed into CSV for later use.

**Anime record columns:**
- Anime name
- Episode that I watched until
- Full URL
- Page title
- Website

In [ ]:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA
from google.colab import drive
drive.mount('/content/drive')

#drive.flush_and_unmount()

In [ ]:
# !cat '/content/drive/MyDrive/Colab Notebooks/mal-anime-query/Data/onetab-list.txt'
data_dir = '/content/drive/MyDrive/Colab Notebooks/mal-anime-query/Data/'

In [ ]:
import re
import uuid

class AnimeRecord:
  def __init__(self, url, page_title, name, episode, website, id):
    self.id = id
    self.page_title = page_title
    self.url = url
    self.name = name
    self.episode = episode
    self.website = website

  @classmethod
  def construct(self, url, page_title):
    id = uuid.uuid4()
    name = ''
    episode = 0
    website = ''

    clean_page_title = page_title
    match = re.search('Episode (?P<episode>[\d]+)', page_title)
    if match:
        episode = match.group('episode').strip()
        clean_page_title = page_title[:match.span()[0]]

    match = re.search('(Watch )*(?P<title>.*)', clean_page_title)
    if match:
      name = match.group('title').strip()

    match = re.search('(http[s]*):\/\/([\w\d]+\.)?(?P<site>[\w\d]+\.[\w]+)\/.*', url)
    if match:
      website = match.group('site').strip()

    return self(url=url, page_title=page_title, id=id, name=name, episode=episode, website=website)

  @classmethod
  def from_dict(self, dict):
    return self(url=dict['url'], page_title=dict['page_title'], id=dict['id'], name=dict['name'], episode=dict['episode'], website=dict['website'])

  def __str__(self):
    return "Name='%s', Episode='%s', Website='%s'" % (self.name, self.episode, self.website)

In [ ]:
import re

input_file = data_dir + 'onetab-raw2.txt'

animes = []
i = 0
with open(input_file, 'r') as f:
  lines = f.readlines()

  for line in lines:
    # Remove possible irrelevant lines
    if not re.match('http[s]?:\/\/.*anime.*\.[a-z]+\/.* \|', line): # Assumption all anime sites has 'anime' in url
      continue
    if 'myanimelist' in line: # Ignore myanimelist
      continue
    if not len(line.strip()):
      continue

    separator_index = line.index('|')
    page_url = line[0:separator_index].strip()
    page_title = line[separator_index+1:].strip()

    anime = AnimeRecord.construct(page_url, page_title)
    animes.append(anime)
    
    i += 1
    print('[%d]%s' % (i, anime))

[1]Name='Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara', Episode='3', Website='gogoanime.ai'
[2]Name='Osananajimi ga Zettai ni Makenai Love Comedy', Episode='2', Website='gogoanime.ai'
[3]Name='Boku no Hero Academia 5th Season', Episode='2', Website='gogoanime.ai'
[4]Name='Hige wo Soru. Soshite Joshikousei wo Hirou.', Episode='3', Website='gogoanime.ai'
[5]Name='Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω', Episode='2', Website='gogoanime.ai'
[6]Name='Sentouin, Hakenshimasu!', Episode='3', Website='gogoanime.ai'
[7]Name='Ijiranaide, Nagatoro-san', Episode='2', Website='gogoanime.ai'
[8]Name='Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita', Episode='2', Website='gogoanime.ai'
[9]Name='Kumo Desu ga, Nani ka?', Episode='14', Website='gogoanime.ai'
[10]Name='Nanatsu no Taizai: Fundo no Shinpan', Episode='13', Website='gogoanime.ai'
[11]Name='Wonder Egg Priority English Subbed Online Free', Episode='0', Website='gogoanime.pro'
[12]Name='Kai

In [ ]:
# Clean up duplicates

animes_dupe_dict = {}
for anime in animes:
  if anime.name not in animes_dupe_dict or animes_dupe_dict[anime.name].episode < anime.episode:
      animes_dupe_dict[anime.name] = anime

i = 0
animes = animes_dupe_dict.values()
for anime in animes:
  i += 1
  print('[%d]%s' % (i, anime))

[1]Name='Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara', Episode='3', Website='gogoanime.ai'
[2]Name='Osananajimi ga Zettai ni Makenai Love Comedy', Episode='2', Website='gogoanime.ai'
[3]Name='Boku no Hero Academia 5th Season', Episode='2', Website='gogoanime.ai'
[4]Name='Hige wo Soru. Soshite Joshikousei wo Hirou.', Episode='3', Website='gogoanime.ai'
[5]Name='Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω', Episode='2', Website='gogoanime.ai'
[6]Name='Sentouin, Hakenshimasu!', Episode='3', Website='gogoanime.ai'
[7]Name='Ijiranaide, Nagatoro-san', Episode='2', Website='gogoanime.ai'
[8]Name='Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita', Episode='2', Website='gogoanime.ai'
[9]Name='Kumo Desu ga, Nani ka?', Episode='14', Website='gogoanime.ai'
[10]Name='Nanatsu no Taizai: Fundo no Shinpan', Episode='13', Website='gogoanime.ai'
[11]Name='Wonder Egg Priority English Subbed Online Free', Episode='0', Website='gogoanime.pro'
[12]Name='Kai

In [ ]:
print('Possible dirty records:')

for anime in animes:
  if anime.episode == 0:
    print(anime)

Possible dirty records:
Name='Wonder Egg Priority English Subbed Online Free', Episode='0', Website='gogoanime.pro'
Name='Jujutsu Kaisen (TV) English Subbed Online Free', Episode='0', Website='gogoanime.app'
Name='Youjo Senki Movie', Episode='0', Website='4anime.to'
Name='Sword Art Online: Alicization - War of Underworld Anime English Subbed in HD for Free on Animefreak.TV', Episode='0', Website='animefreak.tv'
Name='Kekkai Sensen: Ousama no Restaurant no Ousama OVA Online | English Dubbed-Subbed Episodes', Episode='0', Website='animefreak.tv'
Name='Nanatsu no Taizai OVA 2 Online | English Dubbed-Subbed Episodes', Episode='0', Website='animefreak.tv'


In [ ]:
# Manual clean data

cleaner_map = {
    'Wonder Egg Priority English Subbed Online Free': 'Wonder Egg Priority',
    'Jujutsu Kaisen (TV) English Subbed Online Free': 'Jujutsu Kaisen (TV)',
    'Sword Art Online: Alicization - War of Underworld Anime English Subbed in HD for Free on Animefreak.TV': 'Sword Art Online: Alicization - War of Underworld',
    'Kekkai Sensen: Ousama no Restaurant no Ousama OVA Online | English Dubbed-Subbed Episodes': 'Kekkai Sensen: Ousama no Restaurant no Ousama OVA',
    'Nanatsu no Taizai OVA 2 Online | English Dubbed-Subbed Episodes': 'Nanatsu no Taizai OVA'
}

for anime in animes:
  if anime.name in cleaner_map:
    print('Changing "%s" to "%s"' % (anime.name, cleaner_map[anime.name]))
    anime.name = cleaner_map[anime.name]

Changing "Wonder Egg Priority English Subbed Online Free" to "Wonder Egg Priority"
Changing "Jujutsu Kaisen (TV) English Subbed Online Free" to "Jujutsu Kaisen (TV)"
Changing "Sword Art Online: Alicization - War of Underworld Anime English Subbed in HD for Free on Animefreak.TV" to "Sword Art Online: Alicization - War of Underworld"
Changing "Kekkai Sensen: Ousama no Restaurant no Ousama OVA Online | English Dubbed-Subbed Episodes" to "Kekkai Sensen: Ousama no Restaurant no Ousama OVA"
Changing "Nanatsu no Taizai OVA 2 Online | English Dubbed-Subbed Episodes" to "Nanatsu no Taizai OVA"


## Saving parsed data

Keep the parsed data into persistent file to cache the data for future use.

This also allows us to clean the data before the next step. It's possible that the data is dirty because page title is not standardise to any specific format.

In [ ]:
import csv

output_file = data_dir + 'output.csv'
with open(output_file, mode='w') as csv_file:
    fieldnames = ['id', 'name', 'episode', 'website', 'page_title', 'url']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for anime in animes:
      writer.writerow(vars(anime))

In [ ]:
import csv

i = 0
animes_file = data_dir + 'output.csv'
with open(animes_file, mode='r') as csv_file:
  reader = csv.DictReader(csv_file)
  animes = []
  for row in reader:
    anime = AnimeRecord.from_dict(row)
    animes.append(anime)

    i += 1
    print('[%d]%s' % (i, anime))

[1]Name='Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara', Episode='3', Website='gogoanime.ai'
[2]Name='Osananajimi ga Zettai ni Makenai Love Comedy', Episode='2', Website='gogoanime.ai'
[3]Name='Boku no Hero Academia 5th Season', Episode='2', Website='gogoanime.ai'
[4]Name='Hige wo Soru. Soshite Joshikousei wo Hirou.', Episode='3', Website='gogoanime.ai'
[5]Name='Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω', Episode='2', Website='gogoanime.ai'
[6]Name='Sentouin, Hakenshimasu!', Episode='3', Website='gogoanime.ai'
[7]Name='Ijiranaide, Nagatoro-san', Episode='2', Website='gogoanime.ai'
[8]Name='Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita', Episode='2', Website='gogoanime.ai'
[9]Name='Kumo Desu ga, Nani ka?', Episode='14', Website='gogoanime.ai'
[10]Name='Nanatsu no Taizai: Fundo no Shinpan', Episode='13', Website='gogoanime.ai'
[11]Name='Wonder Egg Priority', Episode='0', Website='gogoanime.pro'
[12]Name='Kaifuku Jutsushi no Yarinaoshi

# Connect to MAL & Query

In [ ]:
!pip install jikanpy

     |████████████████████████████████| 133kB 24.1MB/s 
     |████████████████████████████████| 1.3MB 41.8MB/s 
     |████████████████████████████████| 296kB 41.3MB/s 
     |████████████████████████████████| 143kB 43.7MB/s 


## API References

This section describes some [JikanPy API](https://jikanpy.readthedocs.io/en/latest/) that I would be using. [Jikan's official API documentation](https://jikan.docs.apiary.io/) contains more parameters that may be inserted into JikanPy.

[`jikan.anime()`](https://jikanpy.readthedocs.io/en/latest/#jikanpy.Jikan.anime) gets information on an anime.

Response sample:
```json
{'request_hash': 'request:anime:047ce4420fa843606934309866d292274c149a83', 'request_cached': True, 'request_cache_expiry': 79963, 'mal_id': 457, 'url': 'https://myanimelist.net/anime/457/Mushishi', 'image_url': 'https://cdn.myanimelist.net/images/anime/2/73862.jpg', 'trailer_url': 'https://www.youtube.com/embed/h371H0KIuPo?enablejsapi=1&wmode=opaque&autoplay=1', 'title': 'Mushishi', 'title_english': 'Mushi-Shi', 'title_japanese': '蟲師', 'title_synonyms': [], 'type': 'TV', 'source': 'Manga', 'episodes': 26, 'status': 'Finished Airing', 'airing': False, 'aired': {'from': '2005-10-23T00:00:00+00:00', 'to': '2006-06-19T00:00:00+00:00', 'prop': {'from': {'day': 23, 'month': 10, 'year': 2005}, 'to': {'day': 19, 'month': 6, 'year': 2006}}, 'string': 'Oct 23, 2005 to Jun 19, 2006'}, 'duration': '25 min per ep', 'rating': 'PG-13 - Teens 13 or older', 'score': 8.69, 'scored_by': 208969, 'rank': 49, 'popularity': 171, 'members': 644011, 'favorites': 23588, 'synopsis': '"Mushi": the most basic forms of life in the world. They exist without any goals or purposes aside from simply "being." They are beyond the shackles of the words "good" and "evil." Mushi can exist in countless forms and are capable of mimicking things from the natural world such as plants, diseases, and even phenomena like rainbows. This is, however, just a vague definition of these entities that inhabit the vibrant world of Mushishi, as to even call them a form of life would be an oversimplification. Detailed information on Mushi is scarce because the majority of humans are unaware of their existence. So what are Mushi and why do they exist? This is the question that a "Mushishi," Ginko, ponders constantly. Mushishi are those who research Mushi in hopes of understanding their place in the world\'s hierarchy of life. Ginko chases rumors of occurrences that could be tied to Mushi, all for the sake of finding an answer. It could, after all, lead to the meaning of life itself. [Written by MAL Rewrite]', 'background': None, 'premiered': 'Fall 2005', 'broadcast': 'Sundays at 03:40 (JST)', 'related': {'Adaptation': [{'mal_id': 418, 'type': 'manga', 'name': 'Mushishi', 'url': 'https://myanimelist.net/manga/418/Mushishi'}], 'Sequel': [{'mal_id': 21329, 'type': 'anime', 'name': 'Mushishi: Hihamukage', 'url': 'https://myanimelist.net/anime/21329/Mushishi__Hihamukage'}, {'mal_id': 21939, 'type': 'anime', 'name': 'Mushishi Zoku Shou', 'url': 'https://myanimelist.net/anime/21939/Mushishi_Zoku_Shou'}], 'Summary': [{'mal_id': 39738, 'type': 'anime', 'name': 'Mushishi Recap', 'url': 'https://myanimelist.net/anime/39738/Mushishi_Recap'}]}, 'producers': [{'mal_id': 52, 'type': 'anime', 'name': 'Avex Entertainment', 'url': 'https://myanimelist.net/anime/producer/52/Avex_Entertainment'}, {'mal_id': 82, 'type': 'anime', 'name': 'Marvelous', 'url': 'https://myanimelist.net/anime/producer/82/Marvelous'}, {'mal_id': 147, 'type': 'anime', 'name': 'SKY Perfect Well Think', 'url': 'https://myanimelist.net/anime/producer/147/SKY_Perfect_Well_Think'}, {'mal_id': 711, 'type': 'anime', 'name': 'Delfi Sound', 'url': 'https://myanimelist.net/anime/producer/711/Delfi_Sound'}], 'licensors': [{'mal_id': 102, 'type': 'anime', 'name': 'Funimation', 'url': 'https://myanimelist.net/anime/producer/102/Funimation'}], 'studios': [{'mal_id': 8, 'type': 'anime', 'name': 'Artland', 'url': 'https://myanimelist.net/anime/producer/8/Artland'}], 'genres': [{'mal_id': 2, 'type': 'anime', 'name': 'Adventure', 'url': 'https://myanimelist.net/anime/genre/2/Adventure'}, {'mal_id': 36, 'type': 'anime', 'name': 'Slice of Life', 'url': 'https://myanimelist.net/anime/genre/36/Slice_of_Life'}, {'mal_id': 7, 'type': 'anime', 'name': 'Mystery', 'url': 'https://myanimelist.net/anime/genre/7/Mystery'}, {'mal_id': 13, 'type': 'anime', 'name': 'Historical', 'url': 'https://myanimelist.net/anime/genre/13/Historical'}, {'mal_id': 37, 'type': 'anime', 'name': 'Supernatural', 'url': 'https://myanimelist.net/anime/genre/37/Supernatural'}, {'mal_id': 10, 'type': 'anime', 'name': 'Fantasy', 'url': 'https://myanimelist.net/anime/genre/10/Fantasy'}, {'mal_id': 42, 'type': 'anime', 'name': 'Seinen', 'url': 'https://myanimelist.net/anime/genre/42/Seinen'}], 'opening_themes': ['"The Sore Feet Song" by Ally Kerr'], 'ending_themes': ['#01: "Midori no Za" (緑の座) by Masuda Toshio (ep 1)', '#02: "Mabuta no Hikari" (瞼の光) by Masuda Toshio (ep 2)', '#03: "Yawarakai Kaku" (柔らかい角) by Masuda Toshio (ep 3)', '#04: "Makura Kouji" (枕小路 ) by Masuda Toshio (ep 4)', '#05: "Tabi wo Suru Numa" (旅をする沼) by Masuda Toshio (ep 5)', '#06: "Tsuyu wo Suu Mure" (露を吸う群れ) by Masuda Toshio (ep 6)', '#07: "Ame ga Kuru Niji ga Tatsu" (雨がくる虹がたつ) by Masuda Toshio (ep 7)', '#08: "Unasaka Yori" (海境より)  by Masuda Toshio (ep 8)', '#09: "Omoi Mi" (重い実) by Masuda Toshio (ep 9)', '#10: "Suzuri ni Sumu Shiro" (硯に棲む白) by Masuda Toshio (ep 10)', '#11: "Yama Nemuru" (やまねむる) by Masuda Toshio (ep 11)', '#12: "Sugame no Sakana" (眇の魚) by Masuda Toshio (ep 12)', '#13: "Hitoyobashi" (一夜橋) by Masuda Toshio (ep 13)', '#14: "Kago no Naka" (籠のなか) by Masuda Toshio (ep 14)', '#15: "Haru to Usobuko" (春と嘯く) by Masuda Toshio (ep 15)', '#16: "Akatsuki no Hebi" (暁の蛇) by Masuda Toshio (ep 16)', '#17: "Uromayutori" (虚繭取り) by Masuda Toshio (ep 17)', '#18: "Yama Daku Koromo" (山抱く衣) by Masuda Toshio (ep 18)', '#19: "Tenpen no Ito" (天辺の糸) by Masuda Toshio (ep 19)', '#20: "Fude no Umi" (筆の海) by Masuda Toshio (ep 20)', '#21: "Wataboshi" (綿胞子) by Masuda Toshio (ep 21)', '#22: "Okitsu Miya" (沖つ宮) by Masuda Toshio (ep 22)', '#23: "Sabi no Naku Koe" (錆の鳴く聲) by Masuda Toshio (ep 23)', '#24: "Kagarinokou" (篝野行) by Masuda Toshio (ep 24)', '#25: "Ganpuku Ganka" (眼福眼禍) by Masuda Toshio (ep 25)', '#26: "Kusa wo Fumu Oto" (草を踏む音) by Masuda Toshio (ep 26)'], 'jikan_url': 'https://api.jikan.moe/v3/anime/457', 'headers': {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Wed, 21 Apr 2021 05:34:02 GMT', 'Content-Type': 'application/json', 'Content-Length': '2451', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': '*', 'Cache-Control': 'private, must-revalidate', 'ETag': '"267b5262eebb7ade477a0f5c5590e1a2"', 'X-Request-Hash': 'request:anime:047ce4420fa843606934309866d292274c149a83', 'X-Request-Cached': '1', 'X-Request-Cache-Ttl': '79963', 'Expires': 'Thu, 22 Apr 2021 03:46:45 GMT', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Cache-Status': 'MISS'}}
```

[`jikan.search()`](https://jikanpy.readthedocs.io/en/latest/#jikanpy.Jikan.search) searches for a query on MyAnimeList.

Response sample:
```json
{'request_hash': 'request:search:5fec258ddd152243fe19deb2c52974a575c0a888', 'request_cached': False, 'request_cache_expiry': 432000, 'results': [{'mal_id': 38000, 'url': 'https://myanimelist.net/anime/38000/Kimetsu_no_Yaiba', 'image_url': 'https://cdn.myanimelist.net/images/anime/1286/99889.jpg?s=e497d08bef31ae412e314b90a54acfda', 'title': 'Kimetsu no Yaiba', 'airing': False, 'synopsis': "Ever since the death of his father, the burden of supporting the family has fallen upon Tanjirou Kamado's shoulders. Though living impoverished on a remote mountain, the Kamado family are able to enjo...", 'type': 'TV', 'episodes': 26, 'score': 8.6, 'start_date': '2019-04-06T00:00:00+00:00', 'end_date': '2019-09-28T00:00:00+00:00', 'members': 1637158, 'rated': 'R'}, {'mal_id': 47778, 'url': 'https://myanimelist.net/anime/47778/Kimetsu_no_Yaiba__Yuukaku-hen', 'image_url': 'https://cdn.myanimelist.net/images/anime/1338/111945.jpg?s=8ad61bdf543abb9291fe2eeb52a2cb26', 'title': 'Kimetsu no Yaiba: Yuukaku-hen', 'airing': False, 'synopsis': 'Tanjiro, Zenitsu and Inosuke aided by the Sound Hashira Tengen Uzui travel to Yoshiwara red light district to hunt down a demon that has been terrorizing the town.', 'type': 'TV', 'episodes': 0, 'score': 0, 'start_date': None, 'end_date': None, 'members': 95360, 'rated': 'R'}, {'mal_id': 40456, 'url': 'https://myanimelist.net/anime/40456/Kimetsu_no_Yaiba_Movie__Mugen_Ressha-hen', 'image_url': 'https://cdn.myanimelist.net/images/anime/1704/106947.jpg?s=685b7fa652f5b3df29bd20fc2c8cb32e', 'title': 'Kimetsu no Yaiba Movie: Mugen Ressha-hen', 'airing': False, 'synopsis': "After a string of mysterious disappearances begin to plague a train, the Demon Slayer Corps' multiple attempts to remedy the problem prove fruitless. To prevent further casualties, the flame pillar, K...", 'type': 'Movie', 'episodes': 1, 'score': 8.71, 'start_date': '2020-10-16T00:00:00+00:00', 'end_date': '2020-10-16T00:00:00+00:00', 'members': 408674, 'rated': 'R'}], 'last_page': 20, 'jikan_url': 'https://api.jikan.moe/v3/search/anime?q=Kimetsu no Yaiba&limit=3', 'headers': {'Server': 'nginx/1.18.0 (Ubuntu)', 'Date': 'Wed, 21 Apr 2021 06:00:25 GMT', 'Content-Type': 'application/json', 'Content-Length': '914', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Methods': '*', 'Cache-Control': 'private, must-revalidate', 'ETag': '"a3dd9f1d6f98bd837cfe2b4ab918847d"', 'X-Request-Hash': 'request:search:5fec258ddd152243fe19deb2c52974a575c0a888', 'X-Request-Cached': '', 'X-Request-Cache-Ttl': '432000', 'Expires': 'Mon, 26 Apr 2021 06:00:25 GMT', 'Content-Encoding': 'gzip', 'Vary': 'Accept-Encoding', 'X-Cache-Status': 'MISS'}}
```

In [ ]:
import time
from jikanpy import Jikan
from jikanpy.exceptions import APIException
jikan = Jikan()

# https://jikanpy.readthedocs.io/en/latest/
# time.sleep(4) # Remember to sleep 4 seconds between requests

search_results = {}
for anime in animes:
  try:
    result = jikan.search('anime', anime.name, parameters={ 'limit': 3 })
    search_results[anime.id] = result
    print('Searched "%s", found %d result(s)' % (anime.name, len(result['results'])))
  except APIException as e:
    print('APIException: %s' % e)
  
  time.sleep(4)

Searched "Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara", found 3 result(s)
Searched "Osananajimi ga Zettai ni Makenai Love Comedy", found 3 result(s)
Searched "Boku no Hero Academia 5th Season", found 3 result(s)
Searched "Hige wo Soru. Soshite Joshikousei wo Hirou.", found 3 result(s)
Searched "Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω", found 3 result(s)
Searched "Sentouin, Hakenshimasu!", found 3 result(s)
Searched "Ijiranaide, Nagatoro-san", found 3 result(s)
Searched "Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita", found 3 result(s)
Searched "Kumo Desu ga, Nani ka?", found 3 result(s)
Searched "Nanatsu no Taizai: Fundo no Shinpan", found 3 result(s)
Searched "Wonder Egg Priority", found 3 result(s)
Searched "Kaifuku Jutsushi no Yarinaoshi (Uncensored)", found 3 result(s)
Searched "Jujutsu Kaisen (TV)", found 3 result(s)
Searched "Yuru Camp△ Season 2", found 3 result(s)
Searched "Re:Zero kara Hajimeru Isekai Seikatsu 2nd Season

In [ ]:
import json

search_json_file = data_dir + 'search_results.json'

search_json = json.dumps(search_results, indent=2)
#print(search_json)

with open(search_json_file, mode='w') as f:
  f.write(search_json)

In [ ]:
import json

search_json_file = data_dir + 'search_results.json'

with open(search_json_file, mode='r') as f:
  search_results = json.load(f)

# Map Anime to MAL Id



Search with name doesn't guarantee the accuracy. Therefore, this section will check if the anime name matches the first result using a strict string match.

- If the names match, the result is correct.
- If the names doesn't match, I manually check subsequent result match.1
- If none match, a manual search is required.

In [ ]:
i = 0

anime_dicts = []
for anime in animes:
  search_result = search_results[anime.id]

  matching_result = search_result['results'][0]
  for result in search_result['results']:
    if result['title'] == anime.name:
      matching_result = result

  anime_dict = {
      'anime_id': anime.id,
      'anime_name': anime.name,
      'mal_id': matching_result['mal_id'],
      'mal_name': matching_result['title']
  }
  anime_dicts.append(anime_dict)

  i += 1
  print_text = '[%d]%s -> %s'
  if anime_dict['anime_name'] != anime_dict['mal_name']:
    print_text = '\033[31;47m' + print_text + '\033[0m' # https://en.wikipedia.org/wiki/ANSI_escape_code#SGR_(Select_Graphic_Rendition)_parameters

  print(print_text % (i, anime_dict['anime_name'], anime_dict['mal_name']))

[1]Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara -> Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara
[2]Osananajimi ga Zettai ni Makenai Love Comedy -> Osananajimi ga Zettai ni Makenai Love Comedy
[3]Boku no Hero Academia 5th Season -> Boku no Hero Academia 5th Season
[4]Hige wo Soru. Soshite Joshikousei wo Hirou. -> Hige wo Soru. Soshite Joshikousei wo Hirou.
[5]Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω -> Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω
[6]Sentouin, Hakenshimasu! -> Sentouin, Hakenshimasu!
[7]Ijiranaide, Nagatoro-san -> Ijiranaide, Nagatoro-san
[8]Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita -> Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita
[9]Kumo Desu ga, Nani ka? -> Kumo Desu ga, Nani ka?
[10]Nanatsu no Taizai: Fundo no Shinpan -> Nanatsu no Taizai: Fundo no Shinpan
[11]Wonder Egg Priority -> Wonder Egg Priority
[12]Kaifuku Jutsushi no Yarinaoshi (Uncensored) -> Kaifu

In [ ]:
import csv

output_file = data_dir + 'anime_mapping.csv'
with open(output_file, mode='w') as csv_file:
    fieldnames = ['anime_id', 'anime_name', 'mal_id', 'mal_name']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for anime_dict in anime_dicts:
      writer.writerow(anime_dict)

⚠ Manually clean the anime mapping.

In [ ]:
import csv

animes_file = data_dir + 'anime_mapping_cleaned2.csv'
with open(animes_file, mode='r') as csv_file:
  reader = csv.DictReader(csv_file)
  anime_dicts = []
  for row in reader:
    anime_dicts.append(row)

    i += 1
    print('[%d]%s' % (i, row))

[169]OrderedDict([('anime_id', '02e8ad0c-e2d7-4bab-ab05-c102b86bf318'), ('anime_name', 'Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara'), ('mal_id', '44276'), ('mal_name', 'Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara')])
[170]OrderedDict([('anime_id', '47762650-c973-4085-8fcf-1277d4ac3a28'), ('anime_name', 'Osananajimi ga Zettai ni Makenai Love Comedy'), ('mal_id', '43007'), ('mal_name', 'Osananajimi ga Zettai ni Makenai Love Comedy')])
[171]OrderedDict([('anime_id', 'b7b53d49-4a39-46bd-a477-253daa36f544'), ('anime_name', 'Boku no Hero Academia 5th Season'), ('mal_id', '41587'), ('mal_name', 'Boku no Hero Academia 5th Season')])
[172]OrderedDict([('anime_id', '4252ac74-f103-4825-a37c-3e12186bed11'), ('anime_name', 'Hige wo Soru. Soshite Joshikousei wo Hirou.'), ('mal_id', '40938'), ('mal_name', 'Hige wo Soru. Soshite Joshikousei wo Hirou.')])
[173]OrderedDict([('anime_id', 'dd90eb53-2d3e-425b-8cdd-ad57250583eb'), ('anime_name', 'Isek

# Fetch Anime Details



- **Title**: The full anime title.
- **Genre**: The anime genre (according to MyAnimeList).
- **Season**: The xth season of the anime. Non-season type: OVA, Movie, ONA, Special.
- **Total Episodes**: The total number of episodes that the anime has.
- **Date of Release**: The date when the anime start airing.
- **Date of Completion**: The date when the anime finishes airing.
- **Status**: The airing status of the anime: Unreleased, Ongoing, Released.
- **Last**: The last episode I've watched.

In [ ]:
import time
from jikanpy import Jikan
from jikanpy.exceptions import APIException
jikan = Jikan()

# https://jikanpy.readthedocs.io/en/latest/
# time.sleep(4) # Remember to sleep 4 seconds between requests

anime_results = {}
i = 0
for anime_dict in anime_dicts:
  try:
    result = jikan.anime(anime_dict['mal_id'])
    anime_results[anime_dict['anime_id']] = result
    i += 1
    print('[%d]Get anime [%s -> %s] "%s"' % (i, anime_dict['anime_id'], anime_dict['mal_id'], anime_dict['anime_name']))
  except APIException as e:
    print('APIException: %s' % e)
  
  time.sleep(4)

[1]Get anime [02e8ad0c-e2d7-4bab-ab05-c102b86bf318 -> 44276] "Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara"
[2]Get anime [47762650-c973-4085-8fcf-1277d4ac3a28 -> 43007] "Osananajimi ga Zettai ni Makenai Love Comedy"
[3]Get anime [b7b53d49-4a39-46bd-a477-253daa36f544 -> 41587] "Boku no Hero Academia 5th Season"
[4]Get anime [4252ac74-f103-4825-a37c-3e12186bed11 -> 40938] "Hige wo Soru. Soshite Joshikousei wo Hirou."
[5]Get anime [dd90eb53-2d3e-425b-8cdd-ad57250583eb -> 41623] "Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω"
[6]Get anime [08bb2611-3b94-4133-a785-def7e02b09f1 -> 41456] "Sentouin, Hakenshimasu!"
[7]Get anime [e1a2f1b9-1f40-463f-beb8-27a44a8f7de6 -> 42361] "Ijiranaide, Nagatoro-san"
[8]Get anime [ecc34bc0-d8af-48d6-966a-f925fb0d4bb5 -> 40586] "Slime Taoshite 300-nen, Shiranai Uchi ni Level Max ni Nattemashita"
[9]Get anime [379660f9-a3c7-4374-8b8f-71ae9518e8f8 -> 37984] "Kumo Desu ga, Nani ka?"
[10]Get anime [c58ebe48-924e-4699-85d1-9e1ec839258f

In [ ]:
import json

anime_json_file = data_dir + 'anime_results.json'

anime_json = json.dumps(anime_results, indent=2)
#print(search_json)

with open(anime_json_file, mode='w') as f:
  f.write(anime_json)

In [ ]:
import json

anime_json_file = data_dir + 'anime_results.json'

with open(anime_json_file, mode='r') as f:
  anime_results = json.load(f)

In [ ]:
import csv

final_records = []

for anime in animes:
  result = anime_results[anime.id]

  final_record = {
      'title': result['title'],
      'genre': ', '.join(genre['name'] for genre in result['genres']),
      'total_episode': result['episodes'],
      'release_date': result['aired']['from'],
      'completion_date': result['aired']['to'],
      'last_watched': anime.episode
  }
  final_records.append(final_record)
  print(final_record)

final_output_file = data_dir + 'final_output.csv'
with open(final_output_file, mode='w') as csv_file:
  fieldnames = final_records[0].keys()
  writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

  writer.writeheader()
  for final_record in final_records:
    writer.writerow(final_record)

{'title': 'Kyuukyoku Shinka shita Full Dive RPG ga Genjitsu yori mo Kusoge Dattara', 'genre': 'Action, Game, Comedy, Fantasy', 'total_episode': 12, 'release_date': '2021-04-07T00:00:00+00:00', 'completion_date': None, 'last_watched': '3'}
{'title': 'Osananajimi ga Zettai ni Makenai Love Comedy', 'genre': 'Harem, Comedy, Romance, School', 'total_episode': 12, 'release_date': '2021-04-14T00:00:00+00:00', 'completion_date': None, 'last_watched': '2'}
{'title': 'Boku no Hero Academia 5th Season', 'genre': 'Action, Comedy, Super Power, School, Shounen', 'total_episode': 25, 'release_date': '2021-03-27T00:00:00+00:00', 'completion_date': None, 'last_watched': '2'}
{'title': 'Hige wo Soru. Soshite Joshikousei wo Hirou.', 'genre': 'Drama, Romance', 'total_episode': 13, 'release_date': '2021-04-05T00:00:00+00:00', 'completion_date': None, 'last_watched': '3'}
{'title': 'Isekai Maou to Shoukan Shoujo no Dorei Majutsu Ω', 'genre': 'Harem, Comedy, Magic, Ecchi, Fantasy', 'total_episode': 10, 'rele